# Análise Cruzada de Dados - Aposentados vs Devedores

---

## Bloco 3 - Contexto do Desafio de Negócio

**Caso Agibank: Crédito Consignado**

### **Desafio**: Identificar pessoas presentes em ambas as bases.
**Uma vez que o 'target' desse desafio de análise são aposentados com histórico de dívidas.**

Estratégia:

1. Cruzar bases por CPF

2. Validar identidade por nome

3. Gerar lista qualificada de prospects

In [1]:
import pandas as pd
import numpy as np

# Importando base de dados (arquivo 'aposentadorias.csv') com pandas
aposentadorias_df_original = pd.read_csv('data/aposentadorias.csv', sep=';', encoding='latin-1', skiprows=2)

# Importando base de dados (arquivo 'devedores.csv') com pandas
devedores_df_original = pd.read_csv('data/devedores.csv', sep=';', encoding='latin-1')

---

## Bloco 4 - Padronização de CPFs

**Limpeza e Normalização**

In [2]:
def padronizar_cpf(cpf):
    cpf = cpf.replace('*', 'X').upper()
    return cpf

# Tratamento de valores nulos
devedores_df_original['CPF/CNPJ DEVEDOR'] = devedores_df_original['CPF/CNPJ DEVEDOR'].fillna("")
aposentadorias_df_original["CPF_PADRONIZADO"] = aposentadorias_df_original["CPF"].fillna("")

# Aplicação da padronização
aposentadorias_df_original["CPF_PADRONIZADO"] = aposentadorias_df_original["CPF"].apply(padronizar_cpf)
devedores_df_original["CPF_PADRONIZADO"] = devedores_df_original["CPF/CNPJ DEVEDOR"].apply(padronizar_cpf)

Estratégias de padronização:

- Substituição de asteriscos: `*` → `X` (dados mascarados)

- Maiúsculas: `.upper()` para consistência

- Tratamento de nulos: `.fillna("")` evita erros

---

## Bloco 5 - Identificação de Intersecção

**Encontrando CPFs Comuns**

In [3]:
cpf_iguais = set(aposentadorias_df_original["CPF_PADRONIZADO"]) & set(devedores_df_original["CPF_PADRONIZADO"])
print(cpf_iguais)

{'XXX.883.117-XX', 'XXX.272.114-XX', 'XXX.926.247-XX', 'XXX.182.124-XX', 'XXX.874.467-XX', 'XXX.013.866-XX', 'XXX.031.855-XX', 'XXX.290.777-XX', 'XXX.337.338-XX', 'XXX.823.827-XX', 'XXX.454.727-XX', 'XXX.163.608-XX', 'XXX.642.021-XX', 'XXX.734.197-XX', 'XXX.037.727-XX', 'XXX.461.231-XX', 'XXX.573.307-XX', 'XXX.114.148-XX', 'XXX.952.137-XX', 'XXX.982.677-XX', 'XXX.887.307-XX', 'XXX.301.821-XX', 'XXX.633.211-XX', 'XXX.976.541-XX', 'XXX.956.587-XX'}


Conceitos aplicados:

- `set()`: Converte lista em conjunto (remove duplicatas)

- `&` (intersecção): Elementos presentes em ambos os conjuntos

- Eficiência: Operações com sets são muito rápidas

💡 Resultado: Lista de CPFs que aparecem nas duas bases.

---

## Bloco 6 - Filtragem dos Registros Comuns

**Seleção de Registros Relevantes**

In [5]:
# Adicionando coluna CPF_PADRONIZADO às tabelas aposentados e devedores 
aposentados_comuns = aposentadorias_df_original[aposentadorias_df_original["CPF_PADRONIZADO"].isin(cpf_iguais)]
devedores_comuns = devedores_df_original[devedores_df_original["CPF_PADRONIZADO"].isin(cpf_iguais)]

In [8]:
# Visualização inicial
aposentados_comuns[["CPF_PADRONIZADO", "Nome"]].head()

,CPF_PADRONIZADO,Nome
99,XXX.874.467-XX,AIDA AMORIM DE AZEREDO
472,XXX.573.307-XX,ANTONIO JOSE SOARES DE OLIVEIRA
798,XXX.887.307-XX,CARMEN LUCIA M M GUTTMMAN BICHO
980,XXX.976.541-XX,DAISY MARIA GUSMAO WELLISCH
988,XXX.952.137-XX,DALTON LEAL DIMA


In [9]:
devedores_comuns[["CPF_PADRONIZADO", "NOME DO DEVEDOR"]].head()

,CPF_PADRONIZADO,NOME DO DEVEDOR
1203,XXX.883.117-XX,Nuno Vieira Leal
1304,XXX.887.307-XX,Carmen Lucia Meireles Maia Guttmman Bicho
1635,XXX.337.338-XX,Jose Aparecido de Souza
1740,XXX.461.231-XX,Francisco Machado Carneiro
1826,XXX.114.148-XX,Raimundo Nonato de Farias


Método .isin():

- Filtro por lista: Seleciona apenas linhas onde CPF está na intersecção

- Eficiência: Mais rápido que múltiplos `==`

- Flexibilidade: Funciona com qualquer lista de valores

---

## Bloco 7 - Merge/União dos DataFrames

**Combinando Informações**

In [12]:
# Fazer merge/união pelo CPF padronizado 
comparacao = aposentados_comuns.merge(
    devedores_comuns,
    on="CPF_PADRONIZADO",
    how="inner",
)

# Retornando no terminal para comprovação de merge e observação inicial
comparacao[["CPF_PADRONIZADO", "Nome", "NOME DO DEVEDOR"]]

,CPF_PADRONIZADO,Nome,NOME DO DEVEDOR
0,XXX.874.467-XX,AIDA AMORIM DE AZEREDO,Flavio Almeida dos Santos
1,XXX.573.307-XX,ANTONIO JOSE SOARES DE OLIVEIRA,Rafael Abad Sobrinho
2,XXX.887.307-XX,CARMEN LUCIA M M GUTTMMAN BICHO,Carmen Lucia Meireles Maia Guttmman Bicho
3,XXX.976.541-XX,DAISY MARIA GUSMAO WELLISCH,Daisy Maria Gusmao Wellisch
4,XXX.952.137-XX,DALTON LEAL DIMA,Dalton Leal Dima
5,XXX.301.821-XX,DEJAIR CARLOS CARVALHO,Dejair Carlos Carvalho
6,XXX.337.338-XX,ELISA RITSU HONGO,Jose Aparecido de Souza
7,XXX.823.827-XX,ELMO CAVALCANTE GOMES,Elmo Cavalcante Gomes
8,XXX.013.866-XX,EVERALDO JOSE DA SILVA JUNIOR,Everaldo Jose da Silva Junior
9,XXX.290.777-XX,FERNANDO ARTHUR CARVALHO Q DE BARROS,Fernando Arthur Carvalho Queiroz de Barros


Parâmetros do merge:

- `on="CPF_PADRONIZADO"`: Coluna chave para união

- `how="inner"`: Apenas registros presentes em ambas as bases

- Resultado: DataFrame combinado com informações das duas fontes

💡 Tipos de merge: inner, outer, left, right

---

## Bloco 8 - Validação por Similaridade de Nomes

**Função de Verificação Aproximada**

In [13]:
def verificar_nome_aproximado(nome1, nome2, n=6):
    if pd.isna(nome1) or pd.isna(nome2):
        return False

    nome1, nome2 = str(nome1).strip().lower(), str(nome2).strip().lower() # Fazer a padronagem fora da função verificar

    n = min(n, len(nome1), len(nome2))

    inicio_igual = nome1[:n] == nome2[:n]
    fim_igual = nome1[-n:] == nome2[-n:]

    return inicio_igual and fim_igual

Lógica de similaridade:

- Primeiros 6 caracteres: Verificação do início do nome

- Últimos 6 caracteres: Verificação do final (sobrenome)

- Tratamento de nulos: `pd.isna()` evita erros

- Padronização: `.strip().lower()` para comparação consistente

💡 Justificativa: Nomes podem ter grafias ligeiramente diferentes entre bases.

---

## Bloco 9 - Aplicação da Validação

**Processamento em Massa**

In [15]:
# Adicionando coluna verificacao a tabela comparacao
comparacao["verificacao"] = comparacao.apply(
    lambda x: verificar_nome_aproximado(x["Nome"], x["NOME DO DEVEDOR"], n=6),
    axis=1
)

# Retornando no terminal pra comprovação da adição da coluna verificacao a tabela comparacao
comparacao[["CPF_PADRONIZADO", "Nome", "NOME DO DEVEDOR", "verificacao"]].head()

,CPF_PADRONIZADO,Nome,NOME DO DEVEDOR,verificacao
0,XXX.874.467-XX,AIDA AMORIM DE AZEREDO,Flavio Almeida dos Santos,False
1,XXX.573.307-XX,ANTONIO JOSE SOARES DE OLIVEIRA,Rafael Abad Sobrinho,False
2,XXX.887.307-XX,CARMEN LUCIA M M GUTTMMAN BICHO,Carmen Lucia Meireles Maia Guttmman Bicho,True
3,XXX.976.541-XX,DAISY MARIA GUSMAO WELLISCH,Daisy Maria Gusmao Wellisch,True
4,XXX.952.137-XX,DALTON LEAL DIMA,Dalton Leal Dima,True


Método `.apply()`:

- `lambda`: Função anônima para cada linha

- `axis=1`: Aplica por linha (não por coluna)

- Resultado: Nova coluna booleana com resultado da validação

---

## Bloco 10 - Filtragem Final e Contagem

**Lista Final de Prospects**

In [18]:
# Filtragem que retorna apenas as linhas onde os dados da coluna verificacao são iguais a True
possiveis_emprestimos_consignados = comparacao[comparacao["verificacao"] == True]

# Setando colunas de interesse para dataframe de possiveis_emprestimos_consignados
possiveis_emprestimos_consignados = possiveis_emprestimos_consignados[["CPF_PADRONIZADO", "Nome", "NOME DO DEVEDOR", "verificacao"]]

# Retornando no terminal
possiveis_emprestimos_consignados

,CPF_PADRONIZADO,Nome,NOME DO DEVEDOR,verificacao
2,XXX.887.307-XX,CARMEN LUCIA M M GUTTMMAN BICHO,Carmen Lucia Meireles Maia Guttmman Bicho,True
3,XXX.976.541-XX,DAISY MARIA GUSMAO WELLISCH,Daisy Maria Gusmao Wellisch,True
4,XXX.952.137-XX,DALTON LEAL DIMA,Dalton Leal Dima,True
5,XXX.301.821-XX,DEJAIR CARLOS CARVALHO,Dejair Carlos Carvalho,True
7,XXX.823.827-XX,ELMO CAVALCANTE GOMES,Elmo Cavalcante Gomes,True
8,XXX.013.866-XX,EVERALDO JOSE DA SILVA JUNIOR,Everaldo Jose da Silva Junior,True
9,XXX.290.777-XX,FERNANDO ARTHUR CARVALHO Q DE BARROS,Fernando Arthur Carvalho Queiroz de Barros,True
10,XXX.163.608-XX,FRANCISCO CARLOS SERRANO,Francisco Carlos Serrano,True
11,XXX.461.231-XX,FRANCISCO MACHADO CARNEIRO,Francisco Machado Carneiro,True
12,XXX.982.677-XX,IVAN RAMOS CASTRO,Ivan Ramos Castro,True


In [19]:
# Contando quantas linhas iguais as True existem na coluna verificacao
qtd_true = comparacao["verificacao"].sum() # Como True é interpretado como 1 pelo Python, o .sum() conta os True

print(f"Total de possíveis empréstimos consignados: {qtd_true}")

Total de possíveis empréstimos consignados: 20


Truque do `.sum()`:

- Boolean como número: `True` = 1, `False` = 0

- Contagem automática: `.sum()` conta quantos True existem

- Eficiência: Mais rápido que `.count()` com filtro

---